In [ ]:
import pandas as pd

In [ ]:
import os
os.environ['PATH'] = "/usr/local/sbin:/usr/local/bin:/usr/sbin:/opt/R/4.2.3/bin:/usr/bin:/sbin:/bin:/opt/miniconda3/conda/bin:/opt/miniconda3/bin"
%load_ext rpy2.ipython

In [ ]:
#! git clone https://github.com/slagtermaarten/genesets.git ~/libs/genesets/
#! git clone https://github.com/slagtermaarten/maartenutils.git ~/libs/maartenutils/

In [ ]:
#%%R
#if (!require(genesets)) {
#  devtools::install('~/libs/genesets')
#}

# if (!require(genesets)) {
#   devtools::install('~/libs/maartenutils')
# }

In [ ]:
%%R
library(magrittr)
library("data.table")
library(edgeR)
library(flexgsea)
library(foreach)
library(doParallel)

In [ ]:
%%R
devtools::load_all('~/libs/genesets')
source("https://raw.githubusercontent.com/slagtermaarten/TONIC/refs/heads/master/R/GSEA_funcs_paired.R")
#source("https://github.com/slagtermaarten/maartenutils/blob/master/R/datatable_merging.R")

In [ ]:
%%R
devtools::load_all('~/libs/maartenutils')

In [ ]:
%%R
source("https://raw.githubusercontent.com/slagtermaarten/TONIC/refs/heads/master/R/misc.R")
timepoints <- c('baseline' = 'Baseline', 
  'post.induction' = 'Post-induction', 'on.nivo' = 'On nivo')
patient_labels <- readRDS('patient_labels.rds')

In [ ]:
%%R
mut_load_var_types <- 
 c('Missense Variant', 'Frameshift Variant', 'Stop Gained', 
   'Conservative Inframe Insertion', 'Conservative Inframe Deletion', 
   'Disruptive Inframe Insertion', 'Disruptive Inframe Deletion',
   'Structural Interaction Variant', 
   # 'TF Binding Site Variant', 
   'Stop Lost', 
   # 'Start Lost', 
   'Protein Protein Contact', 'Stop Retained Variant') %>% sort
# tolower(paste(mut_load_var_types, collapse = ', '))

timepoints <- c('baseline' = 'Baseline', 
  'post.induction' = 'Post-induction', 'on.nivo' = 'On nivo')
it_timepoints <- timepoints
timepoints_inv <- setNames(names(timepoints), timepoints)

## Labels for timepoints 1, 2 and 3
timepoint_labels <- c('baseline', 'post_induction', 'on_nivo')

blood_timepoints <- auto_name(c(-2, 0, 6, 10, 12))
blood_timepoints_subs <- setNames(blood_timepoints, 
                                  c(timepoints, 'On nivo 2', 'On nivo 3'))
blood_timepoints_subs_inv <- setNames(c(timepoints, 'On nivo 2', 'On nivo 3'),
                                      blood_timepoints)
blood_timepoint_names <-
  setNames(paste0('PBL ', c('Baseline', 'Post-induction', 'On nivo', 
                  'On nivo + 4', 'On nivo + 6')), blood_timepoints)


## Marleen's ingrained way of ordering the projects
treatment_arms <- c('Radiotherapy', 'Doxorubicin', 'Cyclophosphamide',
                    'Cisplatin', 'No induction')
## Ordering by clinical benefit and NanoString results
treatment_arms <- c('No induction', 'Radiotherapy', 'Cyclophosphamide',
                    'Cisplatin', 'Doxorubicin')
label_reps <- c('efron_thisted_estimator' = 'TCRSeq repertoire size', 
                'adaptive_t_cells' = 'TCRSeq %T-cells',
                'sample_clonality' = 'TCRSeq clonality',
                'ca15_3' = 'Baseline CA15.3',
                's_til' = 'Baseline stromal TIL [%]',
                'pd_l1_immunoinfiltrate' = 'Baseline PD-L1 [%]')

In [ ]:
ann = pd.read_csv('../../processed_data/tonic_final_not_full_metadata_response_add_sets.csv', sep=',', index_col=0)
two_tp = [x for x in ann[ann.Timepoint == 'Postinduction'].StudyID.values.tolist() if x in ann[ann.Timepoint == 'Baseline'].StudyID.values.tolist()]
ann_two_tp = ann[ann.StudyID.isin(two_tp)]

In [ ]:
ann_two_tp = ann_two_tp[(ann_two_tp.Cohort == 'T1_1') | (ann_two_tp.Induction != 'Control')]

In [ ]:
rna_sample_annotation = ann_two_tp.reset_index().rename(columns = {'Timepoint': 'timepoint', 'index': 'cf_number', 'StudyID': 'patient', 'Induction': 'arm', 'Response': 'clinical_response'})
rna_sample_annotation = rna_sample_annotation.loc[:, ['patient', 'cf_number', 'timepoint', 'arm', 'clinical_response']].replace({'timepoint': {'Postinduction': 'Post-induction', 'On-nivo': 'On nivo'}, 
                                                                                                                                'arm': {'Control': 'No induction'}})
rna_sample_annotation = rna_sample_annotation[rna_sample_annotation.timepoint != 'On_nivo']
rna_sample_annotation = rna_sample_annotation[rna_sample_annotation.arm.isin(['No induction', 'Doxorubicin', 'Cisplatin'])]
%R -i rna_sample_annotation

In [ ]:
rna_sample_annotation.arm.value_counts()

In [ ]:
%%R
rna_sample_annotation <- as.data.table(rna_sample_annotation, keep.rownames = T)
rna_sample_annotation$timepoint <- factor(rna_sample_annotation$timepoint)
rna_sample_annotation$arm <- factor(rna_sample_annotation$arm)
local_run <- F

In [ ]:
%%R
library(readxl)
library(naturalsort)
data_dir = './'


rna_sample_annotation[, timepoint :=
                      factor(timepoints[as.integer(timepoint)],
                             levels = timepoints)]

setkey(rna_sample_annotation, patient)

rna_sample_annotation <-
  rna_sample_annotation[naturalsort::naturalorder(patient)]
rna_sample_annotation[, cf_number := tolower(cf_number)]


In [ ]:
tmm = pd.read_csv('../../processed_data/TMM_counts_all_TONIC_batch_corrected.csv',
                         index_col=0)
%R -i tmm

In [ ]:
%%R
tmm <- as.data.table(tmm, keep.rownames = T)
rownames(tmm) <- tmm$rn
tmm[ , rn:=NULL]
colnames(tmm) <- tolower(colnames(tmm))

In [ ]:
%%R
gsea_all_arms <- function(
  gene_sets = HALLMARK_pathways,
  patients = rna_sample_annotation[!is.na(clinical_response), unique(patient)],
  allowed_timepoints = c('Baseline', 'Post-induction'),
  gene_score_fn = my_unpaired_WC_test,
  exp_mat = rna_read_counts_salmon_tmm_M,
  resp_exp = 'timepoint',
  paired_test = F,
  nperm = 1000,
  abs = F,
  fn_extra = '') {

  if (!exists('rna_sample_annotation')) source('R/load_rna_dat.R')

  arm_specific_gsea <-
    lapply(rna_sample_annotation[, levels(arm)], 
      function(l_arm) {
      l_patients <- intersect(rna_sample_annotation[, patient], patients)
      if (l_arm != 'All arms') {
        l_patients <- intersect(l_patients,
          rna_sample_annotation[arm %in% l_arm, patient])
      }
      res <- gsea_wrapper(gene_sets = gene_sets,
        patients = l_patients,
        allowed_timepoints = allowed_timepoints,
        gene_score_fn = gene_score_fn,
        exp_mat = exp_mat,
        resp_exp = resp_exp,
        paired_test = paired_test,
        nperm = nperm,
        abs = abs,
        fn_extra = fn_extra)
      if (!is.null(res)) 
        res <- cbind(res$gsea, 'arm' = l_arm)
      return(res)
    }) %>% rbindlist(fill = T)

  if (!null_dat(arm_specific_gsea)) {
    arm_specific_gsea$arm <- factor(arm_specific_gsea$arm,
      levels = treatment_arms)
  }

  return(arm_specific_gsea)
}

In [ ]:
%%R
danaher_sig = read_gmt("/home/m.chelushkin/Michiel_projects/rna_seq/signatures/danaher_pathways.gmt")
cc_sig = read_gmt("/home/m.chelushkin/Michiel_projects/rna_seq/cancer_cell/gene_signatures.gmt")

In [ ]:
%R -o cc_sig

In [ ]:
%%R
registerDoParallel(cores=28)
res_cis <- gsea_wrapper(gene_sets = cc_sig,
  patients = rna_sample_annotation[arm == 'Cisplatin', unique(patient)],
  gene_score_fn = my_paired_WC_test,
  paired_test = T,
allowed_timepoints =  c('Baseline', 'Post-induction'),
  resp_exp = 'timepoint',
  exp_mat = tmm,
  nperm = 1000,
  abs = F,
  fn_extra = '')

In [ ]:
%R -o res_cis
res_cis['gsea'].assign(arm = 'Cisplatin').to_csv('../../processed_data/flexgsea_new_cohorts_bagaev_Cisplatin.tsv', sep='\t')

In [ ]:
%%R
registerDoParallel(cores=28)
res_dox <- gsea_wrapper(gene_sets = cc_sig,
  patients = rna_sample_annotation[arm == 'Doxorubicin', unique(patient)],
  gene_score_fn = my_paired_WC_test,
  paired_test = T,
allowed_timepoints =  c('Baseline', 'Post-induction'),
  resp_exp = 'timepoint',
  exp_mat = tmm,
  nperm = 1000,
  abs = F,
  fn_extra = '')

In [ ]:
%R -o res_dox
res_cis['gsea'].assign(arm = 'Doxorubicin').to_csv('../../processed_data/flexgsea_new_cohorts_bagaev_Doxorubicin.tsv', sep='\t')

In [ ]:
%%R
cc_sig[['Th2_signature']] <- NULL

In [ ]:
%%R
registerDoParallel(cores=36)
res_noi3 <- gsea_wrapper(gene_sets = cc_sig,
  patients = rna_sample_annotation[arm == 'No induction', unique(patient)],
  gene_score_fn = my_paired_WC_test,
  paired_test = T,
allowed_timepoints =  c('Baseline', 'Post-induction'),
  resp_exp = 'timepoint',
  exp_mat = tmm,
  nperm = 1000,
  abs = F,
  fn_extra = '')

In [ ]:
%R -o res_noi3
res_noi3['gsea'].assign(arm = 'No induction').to_csv('../../processed_data/flexgsea_new_cohorts_bagaev_Noind_Monday.tsv', sep='\t')